# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

<b>Описание данных:</b><br>
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. <br>
Известно:<br>

* сalls — количество звонков,<br>
* minutes — суммарная длительность звонков в минутах,<br>
* messages — количество sms-сообщений,<br>
* mb_used — израсходованный интернет-трафик в Мб,<br>
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).<br>

## Откройте и изучите файл

Скомпонуем все импорты в первом блоке. <br>
Последние два импорта нужны для дополнительного задания.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Как и было сказано в условии задачи, предобработка данных не понадобится — её уже сделали.<br>

In [3]:
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


## Разбейте данные на выборки

Указываем признаки и целевой признак для всех выборок.

In [4]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

Делим датасет на тренировочную часть и "оставшуюся".

In [5]:
features_train, features_remaining, target_train, target_remaining = train_test_split(features,target, train_size=0.6, random_state = 123456)

А теперь разделяем оставшуюся часть на валидационную и тестовую.

In [6]:
features_valid, features_test, target_valid, target_test = train_test_split(features_remaining,target_remaining, test_size=0.5, random_state = 123456)

Проверим размеры.

In [7]:
print("Обучающая выборка:", features_train.shape, target_train.shape)
print("Валидационная выборка:", features_valid.shape, target_valid.shape)
print("Тестовая выборка:", features_test.shape, target_test.shape)

Обучающая выборка: (1928, 4) (1928,)
Валидационная выборка: (643, 4) (643,)
Тестовая выборка: (643, 4) (643,)


## Исследуйте модели

Изучим модель DecisionTreeClassifier, изменяя гиперпараметр глубины дерева.

In [8]:
best_model = None
best_result = 0
for depth in range(1, 12):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # выбор модели
    model.fit(features_train, target_train) # обучение модель
    predictions = model.predict(features_valid) # получения предсказания для модели
    result = accuracy_score(target_valid, predictions) # считаем качество модели
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        
print('Лучший результат:', best_result)
print("Глубина:", best_depth)

Лучший результат: 0.7978227060653188
Глубина: 5


Каждая итерация анализа может показать разный лучший результат.<br>
В первый раз лучший результат оказался при глубине деревна 6, в следующий - 9.<br>

Теперь исследуем модель RandomTreeClassifier, изменяя гиперпараметры глубины и количества оценщиков.

In [9]:
best_model2 = None
best_result2 = 0
for est in range(10, 51, 5):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state = 12345, n_estimators=est, max_depth = depth)
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions)
        if result > best_result2:
            best_model2 = model
            best_result2 = result
            best_est = est
            best_depth = depth
print('Лучший результат:', best_result2)
print("Глубина:", best_depth)
print("Количество оценщиков:", best_est)

Лучший результат: 0.7807153965785381
Глубина: 1
Количество оценщиков: 10


Выберем лучшую модель.

In [10]:
if best_result2 > best_result:
    best_model = best_model2

В обоих случаях значение точности приближалось к 80%. Что, на мой взгляд, неплохо, но оставляет желать лучшего.<br>
Но лучший результат показала всё-таки модель DecisionTreeClassifier.<br>
Даже после нескольких итераций.

## Проверьте модель на тестовой выборке

In [11]:
predictions_test = best_model.predict(features_test)

In [12]:
result_test = accuracy_score(predictions_test, target_test)
result_test

0.8102643856920684

И на тестовой выборке значение точности приближается к 80%. Что соответствует условиям задачи о доле правильных ответов в 0.75.

## (бонус) Проверьте модели на адекватность

Для оценки адекватности я решил проверить другие метрики, примениемые к данной задаче классификации.

Precision можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными, а recall показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.

In [13]:
precision_score(predictions_test, target_test)

0.4245810055865922

In [14]:
recall_score(predictions_test, target_test)

0.8

F-мера — среднее гармоническое precision и recall. Стремится к максимуму при precision и recall, близких к 1.

In [15]:
f1_score(predictions_test, target_test)

0.5547445255474452

По этим метрикам модель показала довольно скудный результат.

Теперь проверим модель, сравнив с константной, с помощью DummyClassifier.

In [16]:
dummy = DummyClassifier(strategy='most_frequent').fit(features_train, target_train)
dummy_pred = dummy.predict(features_test)
print('Точность: ', accuracy_score(target_test, dummy_pred))

Точность:  0.7216174183514774


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75


## Выводы

В результате выполнения проекта построена модель для рекомендации наиболее подходящего тарифа оператора "Мегалайн" с учётом данных о поведении клиентов.<br>
Ислеедованы модели DecisionTreeClassifier с изменением параметра глубины древа и RandomTreeClassifier с изменением параметров глубины древа и количества оценщиков.<br>

В результате лучшую точность показала модель DecisionTreeClassifier.<br>
Точность модели на валидационной и тестовых выборках приблизительно равна 0.8.<br>

Проверены также другие метрики - precision ~ 0.5, recall ~ 0.82 и F-1 ~0.62. <br>

Для проверки модели на адекватность построена dummy-модель со стратегией "most_frequent", показавшая меньшую точность, чем DecisionTreeClassifier.